In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Load Data

In [ ]:
# Load raw data
df = pd.read_csv('../data/raw/player_data.csv')
df['date'] = pd.to_datetime(df['date'])
df['registration_date'] = pd.to_datetime(df['registration_date'])

print(f"Dataset shape: {df.shape}")
print(f"Total players: {df['player_id'].nunique():,}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

df.head()

## 2. Basic Statistics

In [ ]:
# Get latest data for each player
df_latest = df.sort_values('date').groupby('player_id').last().reset_index()

print("Overall Statistics:")
print(f"Total Players: {len(df_latest):,}")
print(f"Churned Players: {df_latest['churned'].sum():,}")
print(f"Churn Rate: {df_latest['churned'].mean():.2%}")
print(f"\nPlayer Archetype Distribution:")
print(df_latest['archetype'].value_counts())

## 3. Churn Rate by Segment

In [ ]:
# Churn by archetype
churn_by_archetype = df_latest.groupby('archetype')['churned'].agg(['mean', 'count'])
churn_by_archetype.columns = ['Churn Rate', 'Player Count']
churn_by_archetype['Churn Rate'] = churn_by_archetype['Churn Rate'] * 100

fig = px.bar(
    churn_by_archetype.reset_index(),
    x='archetype',
    y='Churn Rate',
    title='Churn Rate by Player Archetype',
    text='Churn Rate',
    color='Churn Rate',
    color_continuous_scale='RdYlGn_r'
)
fig.update_traces(texttemplate='%{text:.1f}%')
fig.show()

## 4. Feature Distributions

In [ ]:
# Key features to analyze
features = [
    'sessions_per_week',
    'avg_session_length_mins',
    'days_since_last_login',
    'win_rate',
    'friends_online',
    'monthly_spent'
]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, feature in enumerate(features):
    # Plot distributions for churned vs not churned
    df_latest[df_latest['churned'] == 0][feature].hist(
        ax=axes[idx], alpha=0.5, label='Not Churned', bins=30, color='green'
    )
    df_latest[df_latest['churned'] == 1][feature].hist(
        ax=axes[idx], alpha=0.5, label='Churned', bins=30, color='red'
    )
    axes[idx].set_title(feature.replace('_', ' ').title())
    axes[idx].legend()

plt.tight_layout()
plt.show()

## 5. Correlation Analysis

In [ ]:
# Select numeric columns
numeric_cols = df_latest.select_dtypes(include=[np.number]).columns
correlation_matrix = df_latest[numeric_cols].corr()

# Get correlations with churn
churn_correlations = correlation_matrix['churned'].sort_values(ascending=False)

print("Top 10 Features Correlated with Churn:")
print(churn_correlations.head(11)[1:])  # Exclude churned itself

print("\nBottom 10 Features (Negatively Correlated):")
print(churn_correlations.tail(10))

In [ ]:
# Visualize correlation heatmap
selected_features = [
    'sessions_per_week', 'avg_session_length_mins', 'days_since_last_login',
    'win_rate', 'kd_ratio', 'friends_online', 'party_play_percentage',
    'monthly_spent', 'achievements_unlocked', 'churned'
]

plt.figure(figsize=(12, 10))
sns.heatmap(
    df_latest[selected_features].corr(),
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    square=True
)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

## 6. Behavioral Patterns Over Time

In [ ]:
# Average engagement over time
engagement_over_time = df.groupby('date').agg({
    'sessions_per_week': 'mean',
    'avg_session_length_mins': 'mean',
    'monthly_spent': 'mean'
}).reset_index()

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Sessions per Week', 'Session Length', 'Monthly Spending')
)

fig.add_trace(
    go.Scatter(x=engagement_over_time['date'], y=engagement_over_time['sessions_per_week'], name='Sessions'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=engagement_over_time['date'], y=engagement_over_time['avg_session_length_mins'], name='Length'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=engagement_over_time['date'], y=engagement_over_time['monthly_spent'], name='Spending'),
    row=3, col=1
)

fig.update_layout(height=800, title_text="Engagement Metrics Over Time")
fig.show()

## 7. High-Risk Player Profile

In [ ]:
# Compare churned vs non-churned players
comparison = df_latest.groupby('churned').agg({
    'sessions_per_week': 'mean',
    'avg_session_length_mins': 'mean',
    'days_since_last_login': 'mean',
    'friends_online': 'mean',
    'monthly_spent': 'mean',
    'win_rate': 'mean'
}).T

comparison.columns = ['Retained', 'Churned']
comparison['Difference (%)'] = ((comparison['Churned'] - comparison['Retained']) / comparison['Retained'] * 100).round(1)

print("Profile Comparison: Retained vs Churned Players")
print("="*60)
print(comparison)

# Visualize
comparison[['Retained', 'Churned']].plot(kind='bar', figsize=(12, 6))
plt.title('Average Metrics: Retained vs Churned Players')
plt.ylabel('Value')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

## 8. Key Insights

Based on the exploratory analysis:

1. **Churn Rate**: Overall churn rate is around 25%, varying significantly by player type
2. **At-Risk Players**: "at_risk" archetype shows highest churn (40-50%)
3. **Key Indicators**:
   - Low session frequency (<2 per week)
   - Long periods since last login (>14 days)
   - Declining engagement (high decay rate)
   - Limited social connections (<2 friends)
4. **Protective Factors**:
   - Strong social connections
   - Recent purchases
   - High win rate and performance
5. **Monetization Impact**: Spenders are less likely to churn

## Next Steps

1. Feature engineering to create risk scores
2. Model training with XGBoost, Random Forest, LightGBM
3. SHAP analysis for interpretability
4. A/B testing framework for interventions

In [ ]:
# Save summary statistics
summary = {
    'total_players': len(df_latest),
    'churn_rate': df_latest['churned'].mean(),
    'avg_sessions_per_week': df_latest['sessions_per_week'].mean(),
    'avg_monthly_spent': df_latest['monthly_spent'].mean(),
    'high_risk_players': (df_latest['archetype'] == 'at_risk').sum()
}

print("\nDataset Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")